In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv('updated_train.csv')

In [8]:
title_sents=[]
for i in range(train.shape[0]):
    title_sents.append(train.loc[i,'Title'])
    
        
#title_sents

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

#pip install torchtext
import torchtext

import spacy

In [10]:
import string
k=string.punctuation

punct=[]
for i in range(len(k)):
    if(k[i]=='?'):
        continue
    else: 
        punct.append(k[i])
    


In [11]:
spacy_eng=spacy.load('en')
nlp=spacy.load("en_core_web_sm")
def tokenize_eng(text):
    sent=[]
    lema=nlp(text)
    for tok in spacy_eng.tokenizer(text):
        
        word=tok.lemma_
        #word=tok.text
        
        if word not in punct:
            sent.append(word)
    
    return sent

In [12]:

for sent in title_sents:
    sent=tokenize_eng(sent)
    break

In [13]:
title_sents[0]

'Coronavirus (COVID-19) Megathread - News and Updates - 3'

In [14]:
from torchtext.data import Field


TEXT=Field(sequential=True,tokenize=tokenize_eng,lower=True,batch_first=True)
LABEL=Field(sequential=False,use_vocab=False,batch_first=True)

In [19]:

from torchtext.data import TabularDataset

flair_fields=[('Unnamed:0',None),
         ('idx',None),
         ('Title',TEXT),
         ('Score',None),
         ('ID',None),
         ('Image',None),
         ('Num_Comments',None),
         ('Created',None),
         ('Body',None),
         ('Label1Coronovirus',LABEL),
         ('Label2Politics',LABEL),
         ('Label3Non-Political',LABEL),
         ('Label4Objects',LABEL),
         ('Label5AskIndia',LABEL)]
         

In [34]:
train_data_csv=TabularDataset(path='updated_train.csv',
                    format='csv',
                    skip_header=True,
                    fields=flair_fields)

In [81]:
TEXT.build_vocab(train_data_csv,min_freq=1,vectors='fasttext.simple.300d')

.vector_cache/wiki.simple.vec: 293MB [14:26, 338kB/s]                                
100%|█████████▉| 110943/111051 [00:10<00:00, 10857.70it/s]

In [137]:
from torchtext.data import Iterator,BucketIterator

train_iterator=BucketIterator(train_data_csv,
                             batch_size=32,
                             device=-1,
                             sort_key=lambda x:len(x.Title),
                             sort_within_batch=False,
                             repeat=False
                             )

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [138]:
train_data_csv[1].__dict__.keys()

dict_keys(['Title', 'Label1Coronovirus', 'Label2Politics', 'Label3Non-Political', 'Label4Objects', 'Label5AskIndia'])

In [139]:
len(TEXT.vocab)

2266

In [140]:
for example in enumerate(train_iterator):
    print(example[1].Title)
    print(example[1].Label1Coronovirus)
    break

tensor([[2217, 1570, 1272,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1],
        [ 279, 1635, 1567,  175, 1230, 2174,  426, 1956,  325,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1],
        [  93,  428,  822,   50, 1152, 2073,  165,    2,  231, 1074,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1],
        [ 127, 1132,    6,  255,  189, 1908, 1621,    3,    9,   74,    4,  767,
          228,   11,    7, 1499, 1744, 2160,   10,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1],
        [  88, 2104,    5,   81,    2,  470,   64,  181, 1534, 1414,   11, 1439,
          792,  124, 1158,  245, 1178,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    

In [141]:
TEXT.vocab.vectors[100].shape

torch.Size([300])

In [142]:

class BatchWrapper:
    def __init__(self,dat,x_var,*y_vars):
        self.dat=dat
        self.x_vars=x_var
        self.y_vars=y_vars
        
    def __iter__(self):
        for batch in self.dat:
            x=getattr(batch,self.x_vars)
            y=torch.cat([getattr(batch,y).unsqueeze(1) for y in self.y_vars[0]],dim=1).float()
            
            yield(x,y)

    def __len__(self):
        return len(self.dat)
        
train_dat=BatchWrapper(train_iterator,'Title',labels)

In [143]:
a=list(train_data_csv[0].__dict__.keys())
labels=a[1:]

In [144]:
labels

['Label1Coronovirus',
 'Label2Politics',
 'Label3Non-Political',
 'Label4Objects',
 'Label5AskIndia']

In [145]:
next(train_dat.__iter__())

(tensor([[ 134,   25,  200,  ...,    1,    1,    1],
         [ 928, 1978, 1644,  ...,    1,    1,    1],
         [  24,   14,  299,  ...,    1,    1,    1],
         ...,
         [2214,    5,    8,  ...,    1,    1,    1],
         [  28,   37, 1844,  ...,    1,    1,    1],
         [   7,  130, 1003,  ...,    1,    1,    1]]),
 tensor([[0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.],
         [1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.],
         [0., 0

In [146]:
for example in enumerate(train_dat):
    print(example[1][0].shape)
    print(example[1][1].shape)
    print(example[0])
    break
    
#Here y is much smaller 5 dimensional

torch.Size([32, 28])
torch.Size([32, 5])
0


In [147]:
pretrained_vecs=TEXT.vocab.vectors

In [167]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class Model(nn.Module):
    def __init__(self,n_vocab,
                 pretrained_vec,
                batch_size=16,
                embedding_dim=50,
                hidden_dim=64,
                num_layer=2,
                 dropout=0.3,
                 output_dims=5,
                bidirectional=True):
        super(Model,self).__init__()
        self.n_vocab=n_vocab
        self.batch_size=batch_size
        self.embedding_dim=embedding_dim
        self.hidden_dim=hidden_dim
        self.num_layer=num_layer
        self.bidirectional=bidirectional
        self.dropout=dropout
        
        self.output_dims=output_dims
        
        self.embedding=nn.Embedding(n_vocab,embedding_dim)
        self.embedding.weight.data.copy_(pretrained_vec)
        self.embedding.weight.requires_grad=False
        
        
        self.rnn=nn.LSTM(self.embedding_dim, self.hidden_dim,
                       num_layers=self.num_layer,
                       batch_first=True,bidirectional=self.bidirectional,
                        dropout=0.5)
        
        
        
        self.fc1=nn.Linear(hidden_dim*2,self.output_dims)
        
        self.dropout=nn.Dropout(dropout)
        
    def forward(self,data):
        
            
        embedded=self.embedding(data)
        output,(hidden,_)=self.rnn(embedded)
        
        #print(f"Hidden Shape {hidden.shape}")
        
        hidden=torch.cat((hidden[-1,:,:],hidden[-2,:,:]),dim=1)
        
        #print(f"New Hidden Shape {hidden.shape}")
       # print(f"Hidden Dim {hidden.size()}")
        output=F.softmax(self.fc1(hidden))
        
        return output

In [168]:
ENC_EMB_DIM=300
DEC_EMB_DIM=300
HID_DIM=256
ENC_DROPOUT=0.5
DEC_DROPOUT=0.5

Input_Vocab=len(TEXT.vocab)

model=Model(n_vocab=Input_Vocab,pretrained_vec=pretrained_vecs,embedding_dim=ENC_EMB_DIM,hidden_dim=HID_DIM,
           dropout=ENC_DROPOUT)

print(model)

Model(
  (embedding): Embedding(2266, 300)
  (rnn): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (fc1): Linear(in_features=512, out_features=5, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [169]:
import tqdm
import torch.optim as optim
optimizer=optim.Adam(model.parameters())

criterion=nn.BCEWithLogitsLoss()
EPOCH=2

In [170]:
for x,y in tqdm.tqdm(train_dat):
    print(x.shape)
    print(y.shape)
    pred=model(x)
    break
    

a=pred.squeeze(0)
b=y


  0%|          | 0/16 [00:00<?, ?it/s]

torch.Size([32, 50])
torch.Size([32, 5])


In [171]:
epoch_loss_dat=[]

In [178]:
def train(model,train_iter):
    for epoch in range(1,20):
        running_loss=0.0
        running_corrects=0
        model.train()
        epoch_lo=0
        for x,y in tqdm.tqdm(train_iter):
            optimizer.zero_grad()
            #print(x)
            preds=model(x)
            
            #print(f"y: {y.shape}")
            preds=preds.squeeze(0)
            
            #print(f"Preds {preds}")
            
            #print(f"Hello {preds.shape}, {y.shape}")
            for i in range(len(preds)):
                k=preds[i].detach().numpy()
                print(np.argmax(k))
                d=y[i].detach().numpy()
                if(np.argmax(k)==np.argmax(d)):
                    #print(np.argmax(k))
                    epoch_lo+=1
            #print(preds)
            #print(preds.shape)
            loss=criterion(y,preds)
            loss.backward()
            optimizer.step()
            
            running_loss+=loss.item()*x.size(0)
        epoch_loss=running_loss/692
        epoch_loss_dat.append(epoch_loss)
        print(f'Epoch :{epoch}, Training Loss: {epoch_loss}, Num_Correct={epoch_lo}')

In [179]:
train(model,train_dat)



  0%|          | 0/16 [00:00<?, ?it/s]/home/ishan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


4
4
4
0
4
4
0
0
0
4
0
0
4
4
4
4
4
0
4
0
4
4
0
4
4
0
4
4
4
4
4
4




  6%|▋         | 1/16 [00:00<00:05,  2.92it/s]

 12%|█▎        | 2/16 [00:00<00:04,  3.09it/s]

0
4
0
4
4
4
4
4
4
4
4
4
0
4
4
4
4
0
4
0
4
4
4
4
0
4
4
4
4
4
4
4
4
4
4
4
0
4
0
0
4
4
4
0
0
4
4
4
4
4
4
4
4
4
4
4
0
4
4
4
4
4
4
0




 19%|█▉        | 3/16 [00:01<00:05,  2.58it/s]

 25%|██▌       | 4/16 [00:01<00:04,  2.70it/s]

...

0
0
4
0
0
4
0
4
0
0
4
0
0
0
4
0
0
0
0
0
0
4
0
0
0
4
0
0
0
0
0
0




 94%|█████████▍| 15/16 [00:19<00:01,  1.26s/it]

0
0
0
0
4
4
4
0
0
0
0
0
0
4
4
0
4
0
0
0
0
0
0
0
0
0




100%|██████████| 16/16 [00:20<00:00,  1.30s/it]

Epoch :19, Training Loss: 0.5476810766782375, Num_Correct=179
